In [1]:
import pandas as pd
import json
import os
import requests
from gensim.utils import simple_preprocess

In [7]:
# def tokenize(text):

In [2]:
path = './Lyrics_JSON'

In [5]:
# need to remove comments from lyrics

def gather_data(path_to_data):
    data = []
    
    for f in os.listdir(path):
        if os.path.isdir(f) == False:
            if f[-4:] == 'json':
                with open(os.path.join(path, f)) as t:
#                     for lyrics in t['Lyrics']:
                    text = t.read().strip('\n')
                    data.append(str(text))
                    
    return data

In [6]:
lyrics = gather_data(path)

In [7]:
lyrics

['{\n    "Lyrics": [\n        [\n            "[Verse 1]",\n            "Don\'t you know I\'m no good for you?",\n            "I\'ve learned to lose you, can\'t afford to",\n            "Tore my shirt to stop you bleedin\'",\n            "But nothin\' ever stops you leavin\'",\n            "",\n            "[Chorus]",\n            "Quiet when I\'m coming home and I\'m on my own",\n            "I could lie, say I like it like that, like it like that",\n            "I could lie, say I like it like that, like it like that",\n            "",\n            "[Verse 2]",\n            "Don\'t you know too much already?",\n            "I\'ll only hurt you if you let me",\n            "Call me friend, but keep me closer (Call me back)",\n            "And I\'ll call you when the party\'s over",\n            "",\n            "[Chorus]",\n            "Quiet when I\'m coming home and I\'m on my own",\n            "And I could lie, say I like it like that, like it like that",\n            "Yeah, I coul

In [9]:
lyrics2 = {}

for extract in lyrics['Lyrics']:
    lyrics2.append(extract)

TypeError: list indices must be integers or slices, not str